In [85]:
import numpy as np

def rsvd(A, r, p = 8):
    M, N = A.shape
    if M > N:
        raise ValueError('Input matrix should not be "tall"')
    RM = np.random.normal(size = (N, r + p))
    B = A @ RM
    Q, R = np.linalg.qr(B)
    U0, E0, V0 = np.linalg.svd(Q.T @ A);
    UR = Q @ U0
    ER = E0
    VR = V0
    return (UR, ER, VR)
    

def A(i, j):
    return i * j
M = 1024
N = 2048
Ma = np.fromfunction(A, (M, N))
U, E, V = rsvd(Ma, 3)

In [76]:
import numpy as np

def cross(A, eps = 0.1):
    m, n = A.shape
    I = np.arange(m)
    J = np.arange(n)
    r = 0
    i,j = np.unravel_index(np.argmax(np.abs(A), axis=None), A.shape)
    U = np.copy(A[i, :]).reshape(1, n)
    V = np.copy(A[:, j] / A[i, j]).reshape(m, 1)
    I[i] = -1
    J[j] = -1
    B = (A - (V @ U))
    while True:
        r = r + 1
        i = 0
        j = 0
        while I[i] == -1:
            i = i + 1
            if (i == m):
                return V, U
        while J[j] == -1:
            j = j + 1
            if (j == n):
                return V, U
        for jj in range(n):
            if (abs(B[i,jj]) > abs(B[i,j])) and (J[jj] != -1):
                j = jj
        for ii in range(n):
            if (abs(B[ii,j]) > abs(B[i,j])) and (I[ii] != -1):
                i = ii
        
        if abs(B[i, j] * (((m - r)*(n - r)) ** 0.5)) <= eps * np.linalg.norm(V @ U, ord = 'fro'):
            return V, U
        U = np.concatenate((U, B[i, :].reshape(1, n)), axis = 0)
        V = np.concatenate((V, (B[:, j] / B[i,j]).reshape(m, 1)), axis = 1)
        I[i] = -1
        J[j] = -1
        B = (A - (V @ U))
        
        
    
A = np.array([[1, 12, 3], [4, 5, 6], [7, 8, 9]], dtype = np.float64)

U, V = cross(A, 1)
print(A - U @ V)
U, V = cross(A, 0.1)
print(A - U @ V)
U, V = cross(A, 0.01)
print(A - U @ V)

[[0.         0.         0.        ]
 [3.58333333 0.         4.75      ]
 [6.33333333 0.         7.        ]]
[[ 0.          0.          0.        ]
 [-0.71428571  0.          0.        ]
 [ 0.          0.          0.        ]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [8]:
import numpy as np

def hosvd(A):
    N = A.shape[0]
    A1 = np.reshape(A, (N, N * N))
    A2 = np.transpose(A, [1,0,2]).reshape((N, N*N))
    A3 = np.transpose(A, [2,0,1]).reshape((N, N*N))
    print(A.shape, A1.shape, A2.shape, A3.shape)
    U1, E1, V1 = np.linalg.svd(A1)
    U2, E2, V2 = np.linalg.svd(A2)
    U3, E3, V3 = np.linalg.svd(A3)
    G = np.einsum('ai,bj,ck,ijk', U1.T, U2.T, U3.T, A)
    return G, U1, U2, U3
    
A = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
G, U1, U2, U3 = hosvd(A)

(2, 2, 2) (2, 4) (2, 4) (2, 4)


(array([[[-1.42253953e+01,  1.60125603e-02],
         [ 8.28025332e-03,  2.38589095e-01]],
 
        [[ 4.61793060e-03,  5.43770692e-01],
         [ 1.11585148e+00,  2.00114739e-01]]]),
 array([[-0.37616823, -0.92655138],
        [-0.92655138,  0.37616823]]),
 array([[-0.56672424, -0.82390754],
        [-0.82390754,  0.56672424]]),
 array([[-0.64142303, -0.7671874 ],
        [-0.7671874 ,  0.64142303]]))

In [21]:
import numpy as np

def st_hosvd(A):
    N = A.shape[0]
    A1 = np.reshape(A, (N, N * N))
    U1, E, V = np.linalg.svd(A1, full_matrices=False)
    G = np.diag(E) @ V
    G = np.transpose(G.reshape([N, N, N]), [1, 0, 2]).reshape([N, N*N])
    U2, E, V = np.linalg.svd(G, full_matrices=False)
    G = np.diag(E) @ V
    G = np.transpose(G.reshape([N, N, N]), [2, 1, 0]).reshape([N, N*N])
    U3, E, V = np.linalg.svd(G, full_matrices=False)
    G = (np.diag(E) @ V).reshape([N, N, N])
    return G, U1, U2, U3
    
A = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
st_hosvd(A)

(array([[[-1.42253953e+01,  8.28025332e-03],
         [ 4.61793060e-03,  1.11585148e+00]],
 
        [[ 1.60125603e-02,  2.38589095e-01],
         [ 5.43770692e-01,  2.00114739e-01]]]),
 array([[-0.37616823, -0.92655138],
        [-0.92655138,  0.37616823]]),
 array([[-0.56672424, -0.82390754],
        [-0.82390754,  0.56672424]]),
 array([[-0.64142303, -0.7671874 ],
        [-0.7671874 ,  0.64142303]]))

In [13]:
import numpy as np

def TTSVD(A, r):#это TTSVD
    UR = []
    N = A.shape
    NP = 1;
    for i in range(len(N)):
        NP *= N[i]
    NP = NP // N[0]
    A1 = A. reshape (N[0], NP)
    U, s, V = np. linalg .svd(A1 , full_matrices = False )
    UR.append(U[: ,:r[0]])
    G = np.diag(s[:r[0]]) @ V[:r[0], :]
    
    for i in range(len(N) - 2):
        NP = NP // N[i + 1]
        G = G.reshape (r[i] * N[i + 1] , NP)
        U, s, V = np. linalg .svd(G , full_matrices = False )
        UR.append(U[: ,:r[i + 1]].reshape(r[i], N[i + 1], r[i + 1]))
        G = np.diag(s[:r[i + 1]]) @ V[:r[i + 1], :]

    UR.append(G)
    return UR

def TTSVD_3d (A, r1 , r2):#это из учебника для проверки TTSVD только на 3д
    N1 ,N2 ,N3 = A. shape
    A1=A. reshape (N1 , N2 *N3)
    U, s, V = np. linalg .svd(A1 , full_matrices = False )
    G1 = U[: ,: r1] #N1 x r1
    V1 = np.diag(s[: r1 ]) @ V[:r1 , :] # r1 x N2 ∗ N3
    V1 = V1. reshape (r1 * N2 , N3)
    U, s, V = np. linalg .svd(V1 , full_matrices = False )
    G2 = U[: ,: r2] # r1 ∗ N2 x r2
    G2 = G2. reshape (r1 , N2 , r2)# r1 x N2 x r2
    G3 = np.diag(s[: r2 ]) @V [:r2 ,:] # r2 x N3
    return G1 , G2 , G3

A = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

print(TTSVD_3d(A, 2, 2))
print(TTSVD(A, (2, 2)))

(array([[-0.37616823, -0.92655138],
       [-0.92655138,  0.37616823]]), array([[[-0.5645105 ,  0.32808041],
        [-0.82171313, -0.19466474]],

       [[ 0.06461345,  0.754663  ],
        [-0.04405152,  0.53380955]]]), array([[ 9.15252644, 10.94707021],
       [ 0.48089472, -0.40206206]]))
[array([[-0.37616823, -0.92655138],
       [-0.92655138,  0.37616823]]), array([[[-0.5645105 ,  0.32808041],
        [-0.82171313, -0.19466474]],

       [[ 0.06461345,  0.754663  ],
        [-0.04405152,  0.53380955]]]), array([[ 9.15252644, 10.94707021],
       [ 0.48089472, -0.40206206]])]
